In [1]:
import numpy as np
from simulator import popn2_simple_data_morgan

In [ ]:
N = [2500,2500,5000]
T = 200
m=10
length=1e6
L = [0.5,5,1000]
d = popn2_simple_data_morgan(N,T,L,m,length)
d['length']=1000